# Linear Models

In [1]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

In [2]:
import numpy as np
import pandas as pd
from addutils import css_notebook
from sklearn.datasets.samples_generator import make_regression
from sklearn import linear_model
from sklearn import metrics
css_notebook()

In [3]:
import bokeh.plotting as bk
from bokeh import palettes
from bokeh.layouts import gridplot
bk.output_notebook()

Loading BokehJS ...

## 1 Introduction

`LinearRegression` fits a linear model with coefficients $w = (w_1, \ldots, w_p)$ to minimize the residual sum of squares between the observed responses in the dataset, and the responses predicted by the linear approximation.

Mathematically it solves a problem of the form: 
$$ \underset{w}{min} \|Xw -y\|_2^2$$

### 1.1 Simple Linear Regression

Simple linear regression is an approach for predicting a **quantitative response** using a **single feature** (or "predictor" or "input variable"). It has the following mathematical definition:

$y = \beta_0 + \beta_1x$

Where:
- $y$ is the response
- $x$ is the feature
- $\beta_0$ is the intercept
- $\beta_1$ is the coefficient for x

$\beta_0$ and $\beta_1$ are called the **model coefficients**. To build the model, it is necessary to "learn" the values of these coefficients. And once the model has learned these coefficients, it can be used for prediction

### 1.2 Estimating Coefficients

Generally speaking, coefficients are estimated using the **least squares criterion**, which finds the line that minimizes the **sum of squared residuals** (or "sum of squared errors"):

<img src="images/estimating_coefficients.png">

Where:
- The black dots are the **observed values** of x and y.
- The blue line is our **least squares line**.
- The red lines are the **residuals**, which are the distances between the observed values and the least squares line.

How do the model coefficients relate to the least squares line?
- $\beta_0$ is the **intercept** (the value of $y$ when $x$=0)
- $\beta_1$ is the **slope** (the change in $y$ divided by change in $x$)

Here is a graphical representation:

<img src="images/slope_intercept.png">

### 1.3 Interpreting Model Coefficients

How do we interpret the coefficient ($\beta_1$)? A "unit" increase in the variable $x$ is **associated with** $\beta_1$ "unit" increase in variable $y$.
<!-- - Or more clearly: An additional $1,000 spent on TV ads is **associated with** an increase in sales of 47.537 widgets. -->

Note that if an increase in $x$ is associated with a **decrease** in $y$, $\beta_1$ would be **negative**.

In [4]:
X, y = make_regression(n_samples=100000, n_features=1, n_informative=1,
                       random_state=0, noise=35)

In [5]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

fig = bk.figure(plot_width=630, plot_height=300, title=None)
fig.circle(X[::1000, 0], y[::1000],  color='black')
fig.line(X[::1000, 0], regr.predict(X[::1000]), color='red', line_width=3)
fig.grid.grid_line_color = None
fig.axis.minor_tick_out = 0
bk.show(fig)

I have simulated a sine curve (between 60° and 300°) and added some random noise using the following code:

In [6]:
x = np.array([i*np.pi/180 for i in range(60,300,4)])
np.random.seed(10)  #Setting seed for reproducability
y = np.sin(x) + np.random.normal(0,0.15,len(x))
data = pd.DataFrame(np.column_stack([x,y]),columns=['x','y'])
fig = bk.figure(plot_width=630, plot_height=300, title=None)
fig.circle(data['x'], data['y'],)
bk.show(fig)

This resembles a sine curve but not exactly because of the noise. We’ll use this as an example to test different scenarios in this article. Lets try to estimate the sine function using polynomial regression with powers of x form 1 to 15. Lets add a column for each power upto 15 in our dataframe. This can be accomplished using the following code:

In [7]:
for i in range(2,16):  #power of 1 is already there
    data['x_{}'.format(i)] = data['x']**i
print(data.head())

          x         y       x_2       x_3       x_4       x_5       x_6  \
0  1.047198  1.065763  1.096623  1.148381  1.202581  1.259340  1.318778   
1  1.117011  1.006086  1.247713  1.393709  1.556788  1.738948  1.942424   
2  1.186824  0.695374  1.408551  1.671702  1.984016  2.354677  2.794587   
3  1.256637  0.949799  1.579137  1.984402  2.493673  3.133642  3.937850   
4  1.326450  1.063496  1.759470  2.333850  3.095735  4.106339  5.446854   

        x_7       x_8        x_9       x_10       x_11       x_12       x_13  \
0  1.381021  1.446202   1.514459   1.585938   1.660790   1.739176   1.821260   
1  2.169709  2.423588   2.707173   3.023942   3.377775   3.773011   4.214494   
2  3.316683  3.936319   4.671717   5.544505   6.580351   7.809718   9.268760   
3  4.948448  6.218404   7.814277   9.819710  12.339811  15.506664  19.486248   
4  7.224981  9.583578  12.712139  16.862020  22.366630  29.668222  39.353420   

        x_14       x_15  
0   1.907219   1.997235  
1   4.707635   5

In [8]:
def linear_power_model(model, data, power):
    #initialize predictors:
    predictors=['x']
    if power >= 2:
        predictors += ['x_{}'.format(i) for i in range(2,power+1)]
    
    #Fit the model
    model.fit(data[predictors],data['y'])
    y_pred = model.predict(data[predictors])
 
    #Return the result in pre-defined format
    rss = sum((y_pred-data['y'])**2)
    ret = [rss]
    ret.extend([model.intercept_])
    ret.extend(model.coef_)
    return {power: [ret, y_pred]}

In [9]:
results = {}
regr = linear_model.LinearRegression(normalize=True)
for i in range(1,16):
    results.update(linear_power_model(regr, data, i))

In [10]:
grid = []
for m in [1, 3, 6, 9, 12, 15]:
    p = bk.figure(plot_width=230, plot_height=300, title='Plot for power {}'.format(m))
    p.circle(data['x'], data['y'],)
    p.line(data['x'], results[m][1], color='firebrick')
    grid.append(p)
bk.show(gridplot(grid, ncols=3))

## 2 Ridge Regression

Linear Regression rely on the independence of the model terms. When terms are correlated and the columns of the design matrix $X$; have an approximate linear dependence, the matrix $(X^TX)^{-1}$ becomes
close to singular. As a result, the least-squares estimate becomes highly sensitive to random errors in the observed response *y*, producing a large variance. This situation of multicollinearity can arise, for example, when data are collected without an experimental design.

The ridge coefficients minimize a penalized residual sum of squares:

$$ \underset{w}{min} \|Xw -y\|_2^2 + \alpha \|w\|_2^2$$

Here, positive $\alpha  \geq 0 \hspace{2 pt}$ is a complexity parameter that controls the amount of shrinkage: the larger the value of $\alpha$, the greater the amount of shrinkage and thus the coefficients become more robust to collinearity.

In [15]:
results = {}
alpha = 0.1
for i in range(1,16):
    ridgereg = linear_model.Ridge(alpha=alpha,normalize=True)
    results.update(linear_power_model(ridgereg, data, i))

In [16]:
grid = []
for m in [1, 3, 6, 9, 12, 15]:
    p = bk.figure(plot_width=230, plot_height=300, title='Plot for power {}'.format(m))
    p.circle(data['x'], data['y'],)
    p.line(data['x'], results[m][1], color='firebrick')
    grid.append(p)
bk.show(gridplot(grid, ncols=3))

## 3 LASSO

The Lasso is a linear model that estimates sparse coefficients. It is useful in some contexts due to its tendency to prefer solutions with fewer parameter values, effectively reducing the number of variables upon which the given solution is dependent. For this reason, the Lasso and its variants are fundamental to the field of compressed sensing. Under certain conditions, it can recover the exact set of non-zero weights.

Mathematically, it consists of a linear model trained with $\ell_1$ prior as regularizer. The objective function to minimize is:

$$\underset{w}{min\,} { \frac{1}{2n_{samples}} ||X w - y||_2 ^ 2 + \alpha ||w||_1}$$

The lasso estimate thus solves the minimization of the least-squares penalty with $\alpha$ $||w||_1$ added, where $\alpha$ is a constant and $||w||_1$ is the $\ell_1$-norm of the parameter vector.

---

Visit [www.add-for.com](<http://www.add-for.com/IT>) for more tutorials and updates.

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.